# Código Python para buscar publicaçoes Youtube e Gerar os arquivos .csv no formato normalizado e com label 

In [1]:
import os
import pandas as pd
from googleapiclient.discovery import build
import tkinter as tk
from tkinter import ttk

# Função para buscar vídeos
def search_youtube_videos(api_key, query, published_after, published_before, max_results=50):
    youtube = build('youtube', 'v3', developerKey=api_key)
    
    all_videos = []
    next_page_token = None
    
    while True:
        try:
            response = youtube.search().list(
                q=query,
                part='snippet',
                type='video',
                regionCode='BR',
                publishedAfter=published_after,
                publishedBefore=published_before,
                maxResults=max_results,
                pageToken=next_page_token
            ).execute()
        except Exception as e:
            print(f"Erro ao buscar vídeos: {e}")
            raise
        
        for item in response.get('items', []):
            video_id = item['id']['videoId']
            video_response = youtube.videos().list(part='statistics', id=video_id).execute()
            video_stats = video_response['items'][0]['statistics']
            
            if 'commentCount' not in video_stats:
                print(f"Comentários desabilitados para o vídeo: {video_id}")
                continue

            video_data = {
                'ID': video_id,
                'AUTHOR': item['snippet']['channelTitle'],
                'URL': f"https://www.youtube.com/watch?v={video_id}",
                'DATE': item['snippet']['publishedAt'],
                'LIKES': video_stats.get('likeCount', 0),
                'COMMENTS': video_stats.get('commentCount', 0),
                'CONTENT': item['snippet']['description'],
                'SERVICE': 'YouTube',
                'SENTIMENT': 'NEUTRAL',  # Valor padrão, ajustar conforme necessário
                'ENGAGEMENT': 0,   # Valor padrão, ajustar conforme necessário
                'SHARES': 0  # Valor padrão para SHARES
            }
            
            # Adiciona todas as outras informações do item do snippet
            video_data.update(item['snippet'])
            
            all_videos.append(video_data)
        
        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break
    
    return all_videos

# Função para organizar os dados dos vídeos
def organize_video_data(videos):
    df = pd.DataFrame(videos)
    df.columns = df.columns.str.upper()
    return df

# Função para converter sentimento
def convert_sentiment(sentiment_text):
    sentiment_map = {'NEUTRAL': 0, 'POSITIVE': 1, 'NEGATIVE': 2}
    return sentiment_map.get(sentiment_text.upper(), 0)

# Função para salvar os dados em arquivos CSV individuais
def save_videos_to_csv(organized_videos, output_directory):
    for index, video in organized_videos.iterrows():
        video_id = video['ID']
        service = video['SERVICE']
        date = pd.to_datetime(video['DATE']).strftime('%Y%m%d%H%M')
        sentiment = convert_sentiment(video['SENTIMENT'])
        index_formatted = str(index).zfill(7)
    
        filename = f"{service}_{video_id}_{date}_{sentiment}_{index_formatted}.csv"
        filepath = os.path.join(output_directory, filename)
    
        df_single_row = pd.DataFrame([video])
        df_single_row.to_csv(filepath, index=False)

# Função para rodar o código a partir do formulário
def run_code(api_key, query, published_after, published_before):
    videos = search_youtube_videos(api_key, query, published_after, published_before)
    organized_videos = organize_video_data(videos)
    save_videos_to_csv(organized_videos, output_directory)
    print("Dados salvos com sucesso.")

# Configuração do formulário
def create_form():
    def on_submit():
        api_key = api_key_entry.get()
        query = query_entry.get()
        published_after = published_after_entry.get()
        published_before = published_before_entry.get()
        root.destroy()
        run_code(api_key, query, published_after, published_before)

    root = tk.Tk()
    root.title("YouTube Data Fetcher")

    frame = ttk.Frame(root, padding="10")
    frame.grid(row=0, column=0, sticky=(tk.W, tk.E, tk.N, tk.S))

    api_key_label = ttk.Label(frame, text="API Key:")
    api_key_label.grid(row=0, column=0, sticky=tk.W)
    api_key_entry = ttk.Entry(frame, width=30)
    api_key_entry.grid(row=0, column=1, sticky=(tk.W, tk.E))

    query_label = ttk.Label(frame, text="Query:")
    query_label.grid(row=1, column=0, sticky=tk.W)
    query_entry = ttk.Entry(frame, width=30)
    query_entry.grid(row=1, column=1, sticky=(tk.W, tk.E))

    published_after_label = ttk.Label(frame, text="Published After (YYYY-MM-DDTHH:MM:SSZ):")
    published_after_label.grid(row=2, column=0, sticky=tk.W)
    published_after_entry = ttk.Entry(frame, width=30)
    published_after_entry.grid(row=2, column=1, sticky=(tk.W, tk.E))

    published_before_label = ttk.Label(frame, text="Published Before (YYYY-MM-DDTHH:MM:SSZ):")
    published_before_label.grid(row=3, column=0, sticky=tk.W)
    published_before_entry = ttk.Entry(frame, width=30)
    published_before_entry.grid(row=3, column=1, sticky=(tk.W, tk.E))

    submit_button = ttk.Button(frame, text="Submit", command=on_submit)
    submit_button.grid(row=4, column=0, columnspan=2)

    root.mainloop()

# Definir o diretório de saída
output_directory = r'C:\madonna\split'

# Criar e exibir o formulário
create_form()


Comentários desabilitados para o vídeo: -Hyn871iT4g
Comentários desabilitados para o vídeo: VoorkITFyN4
Comentários desabilitados para o vídeo: SDuqpvxdEeo
Comentários desabilitados para o vídeo: b_rETK2Bs5k
Comentários desabilitados para o vídeo: MVng6H0hGj4
Comentários desabilitados para o vídeo: s9bbvvfUVNo
Comentários desabilitados para o vídeo: udyjCbbDK38
Comentários desabilitados para o vídeo: hW_d35HSq68
Dados salvos com sucesso.


# Update de codigo para ajustar entendimento da ultima reunião de 20240613. #